# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="Feedback1.csv"

with open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b'Q01_Best part\tQ02_Worst part\tQ03_Time\n'


### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2019-02-11 15:08:19,529 : INFO : reading file Feedback1.csv...this may take a while
2019-02-11 15:08:19,531 : INFO : read 0 reviews
2019-02-11 15:08:19,630 : INFO : Done reading data file


## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-02-11 15:08:19,642 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-02-11 15:08:19,650 : INFO : collecting all words and their counts
2019-02-11 15:08:19,655 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-11 15:08:19,676 : INFO : collected 2755 word types from a corpus of 32794 raw words and 703 sentences
2019-02-11 15:08:19,680 : INFO : Loading a fresh vocabulary
2019-02-11 15:08:19,694 : INFO : effective_min_count=2 retains 1548 unique words (56% of original 2755, drops 1207)
2019-02-11 15:08:19,695 : INFO : effective_min_count=2 leaves 31587 word corpus (96% of original 32794, drops 1207)
2019-02-11 15:08:19,712 : INFO : deleting the raw counts dictionary of 2755 items
2019-02-11 15:08:19,715 : INFO : sample=0.001 downsamples 77 most-common words
2019-02-11 15:08:19,719 : INFO : downsampling leaves estimated 22207 word corpus (70.3% of prior 31587)
2019-02-11 15:08:19,744 : INFO : estimated required memo

2019-02-11 15:08:20,305 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-02-11 15:08:20,308 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-11 15:08:20,311 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-11 15:08:20,315 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-11 15:08:20,317 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-11 15:08:20,320 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-11 15:08:20,321 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-11 15:08:20,322 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-11 15:08:20,326 : INFO : EPOCH - 2 : training on 32794 raw words (22200 effective words) took 0.0s, 501095 effective words/s
2019-02-11 15:08:20,341 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-02-11 15:08:20,373 : INFO : worker thread f

2019-02-11 15:08:20,845 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-02-11 15:08:20,847 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-02-11 15:08:20,848 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-11 15:08:20,850 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-11 15:08:20,853 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-11 15:08:20,855 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-11 15:08:20,857 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-11 15:08:20,859 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-11 15:08:20,862 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-11 15:08:20,865 : INFO : EPOCH - 10 : training on 32794 raw words (22309 effective words) took 0.1s, 419251 effective words/s
2019-02-11 15:08:20,868 : INFO : training on a 

(221867, 327940)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [17]:

w1 = "loss"
model.wv.most_similar (positive=w1)


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('features', 0.9895908236503601),
 ('function', 0.9875247478485107),
 ('labels', 0.9863935112953186),
 ('hypothesis', 0.9808359146118164),
 ('theoretically', 0.9803018569946289),
 ('dealing', 0.9787272810935974),
 ('suitable', 0.9762242436408997),
 ('explained', 0.973731517791748),
 ('hardest', 0.9730471968650818),
 ('choosing', 0.9728363156318665)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [6]:
# look up top 6 words similar to 'polite'
w1 = ["learn"]
model.wv.most_similar (positive=w1,topn=6)


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('important', 0.9984394311904907),
 ('basics', 0.9983464479446411),
 ('practical', 0.9983209371566772),
 ('gave', 0.9982707500457764),
 ('especially', 0.9979274272918701),
 ('enjoyed', 0.997899055480957)]

In [7]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


KeyError: "word 'france' not in vocabulary"

In [ ]:
# look up top 6 words similar to 'shocked'
w1 = ["help"]
model.wv.most_similar (positive=w1,topn=6)


That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [ ]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [ ]:
# similarity between two different words
model.wv.similarity(w1="matrix",w2="math")

In [ ]:
# similarity between two identical words
model.wv.similarity(w1="matrix",w2="book")

In [ ]:
vec=model.wv['lecture']
print(vec[0:2])

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["matrix","linear","feature"])

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
